# 1

In [12]:
import os, sys; sys.path.append(os.path.dirname(os.getcwd())); ROOT_DIR = '/home/h4zzkr/lab/delOS/'
import pandas as pd
import numpy as np
from pathlib import Path
from core.nlu.semantic_taggers.DefaultTagger.tagger import SemanticTagsExtractor as TaggerBox
from core.nlu.tools.utils import DatasetLoader, encode_token_labels, encode_dataset
from core.nlu.featurizers.transformer_featurizer import SentenceFeaturizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [20]:
dataset_path = os.path.join(ROOT_DIR, 'data/nlu_data/custom')
df_train = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
df_valid = pd.read_csv(os.path.join(dataset_path, 'valid.csv'))
model_path = os.path.join(ROOT_DIR, 'core/nlu/semantic_taggers/DefaultTagger/big_model/')

In [3]:
featurizer = SentenceFeaturizer()

In [4]:
def encode_tokenize(seq, ft):
    encoded = encode_dataset(ft, seq).tolist()
    encoded = ft.encode(
        encoded, output_value='token_embeddings', convert_to_numpy=True, is_pretokenized=True
    )
    return np.array(encoded)

In [26]:
d = DatasetLoader('data/nlu_data/custom', train_intents=False)
df_train, df_valid, intent2id, id2intent, tag2id, id2tag = d.load_prepare_dataset()
validate = True
if df_valid is None:
    validate = False
encoded_seq_train = encode_tokenize(df_train['words'], featurizer)
tag_train = encode_token_labels(df_train["words"], df_train["word_labels"], featurizer, tag2id)

if validate:
    encoded_seq_valid = encode_tokenize(df_valid['words'], featurizer)
    tag_valid = encode_token_labels(
        df_valid["words"], df_valid["word_labels"], featurizer, tag2id)

Loading data...


In [13]:
for i in range(len(id2tag), 72):
    id2tag.update({i : 'O'})

In [14]:
tag_box = TaggerBox(id2tag, path2model=model_path)

In [15]:
epochs = 100
batch_size = 8

opt = Adam(learning_rate=0.001, epsilon=1e-09)
losses = [SparseCategoricalCrossentropy(from_logits=True)]

metrics = [SparseCategoricalAccuracy('accuracy')]
tag_box.model.compile(optimizer=opt, loss=losses, metrics=metrics)

In [16]:
history = tag_box.model.fit(encoded_seq_train,
                        tag_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(encoded_seq_valid, tag_valid), )
#                         callbacks=cp_callback)

Epoch 1/100
7/7 [==============================] - 0s 20ms/step - loss: 0.7103 - accuracy: 0.8913 - val_loss: 0.2061 - val_accuracy: 0.9716
Epoch 2/100
7/7 [==============================] - 0s 8ms/step - loss: 0.2074 - accuracy: 0.9712 - val_loss: 0.1659 - val_accuracy: 0.9716
Epoch 3/100
7/7 [==============================] - 0s 8ms/step - loss: 0.1290 - accuracy: 0.9689 - val_loss: 0.1256 - val_accuracy: 0.9663
Epoch 4/100
7/7 [==============================] - 0s 9ms/step - loss: 0.1145 - accuracy: 0.9677 - val_loss: 0.0998 - val_accuracy: 0.9699
Epoch 5/100
7/7 [==============================] - 0s 9ms/step - loss: 0.0846 - accuracy: 0.9756 - val_loss: 0.0949 - val_accuracy: 0.9699
Epoch 6/100
7/7 [==============================] - 0s 9ms/step - loss: 0.0605 - accuracy: 0.9815 - val_loss: 0.0722 - val_accuracy: 0.9770
Epoch 7/100
7/7 [==============================] - 0s 9ms/step - loss: 0.0527 - accuracy: 0.9823 - val_loss: 0.0628 - val_accuracy: 0.9823
Epoch 8/100
7/7 [=========

In [17]:
text = 'enable lights in the bedroom'
enc_seq, pool_out = featurizer.featurize(text)

In [18]:
tags_classes = tag_box.tag(enc_seq)
tag_ids = np.argmax(tags_classes, 1)[1:-1] # logits of tags
tag_ids

array([6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

# 2

In [1]:
!pip install snips-nlu-parsers

     |████████████████████████████████| 6.8 MB 818 kB/s eta 0:00:01     |███▋                            | 757 kB 818 kB/s eta 0:00:08
     |████████████████████████████████| 829 kB 13.8 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.17.1-py3-none-any.whl size=488732 sha256=26662b7432fc3dd046d951969ca2e1b754c73265dfa3ccf540bc3aeff810c6ca
  Stored in directory: /home/h4zzkr/.cache/pip/wheels/16/4c/84/8a6161d44282ede60ed233d090156c6109a7ab865e49c1c9f6
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.18.2
    Uninstalling future-0.18.2:
      Successfully uninstalled future-0.18.2


In [2]:
from snips_nlu_parsers import BuiltinEntityParser
import json
parser = BuiltinEntityParser.build(language="en")

In [33]:
%time
parsing = parser.parse("Book me restaurant for two people tomorrow")
# print(json.dumps(parsing, indent=2))
parsing

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


[{'value': 'two',
  'range': {'start': 23, 'end': 26},
  'entity': {'kind': 'Number', 'value': 2.0},
  'alternatives': [],
  'entity_kind': 'snips/number'},
 {'value': 'tomorrow',
  'range': {'start': 34, 'end': 42},
  'entity': {'kind': 'InstantTime',
   'value': '2020-07-31 00:00:00 +03:00',
   'grain': 'Day',
   'precision': 'Exact'},
  'alternatives': [],
  'entity_kind': 'snips/date'}]

In [35]:
def translate_entity(items):
    entities_map = {
    'snips/amountOfMoney' : '@amountOfMoney','snips/date' : '@date', \
        'snips/datePeriod' : '@datePeriod','snips/datetime' : '@datetime', \
        'snips/duration' : '@duration', 'snips/number' : '@number', \
        'snips/ordinal' : '@ordinal', 'snips/percentage' : '@percentage', \
        'snips/temperature' : '@temperature', 'snips/time' : '@time', \
        'snips/timePeriod' : '@timePeriod'
    }
    for i in range(len(items)):
        items[i]['entity_kind'] = entities_map[items[i]['entity_kind']]
    return items

In [34]:
%time
translate_entity(parser.parse("Book me restaurant for two people tomorrow"))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


[{'value': 'two',
  'range': {'start': 23, 'end': 26},
  'entity': {'kind': 'Number', 'value': 2.0},
  'alternatives': [],
  'entity_kind': '@number'},
 {'value': 'tomorrow',
  'range': {'start': 34, 'end': 42},
  'entity': {'kind': 'InstantTime',
   'value': '2020-07-31 00:00:00 +03:00',
   'grain': 'Day',
   'precision': 'Exact'},
  'alternatives': [],
  'entity_kind': '@date'}]

In [54]:
from snips_nlu_parsers import BuiltinEntityParser

class BuiltinEntityTagger():
    """
    Huge respect to snips-nlu team for
    https://github.com/snipsco/snips-nlu-ontology
    """
    def __init__(self, lang='en'):
        self.parser = BuiltinEntityParser.build(language="en")
        self.entities = {
        'snips/amountOfMoney' : '@amountOfMoney','snips/date' : '@date', \
            'snips/datePeriod' : '@datePeriod','snips/datetime' : '@datetime', \
            'snips/duration' : '@duration', 'snips/number' : '@number', \
            'snips/ordinal' : '@ordinal', 'snips/percentage' : '@percentage', \
            'snips/temperature' : '@temperature', 'snips/time' : '@time', \
            'snips/timePeriod' : '@timePeriod'
        }
        
    def translate_entity(self, items):
        for i in range(len(items)):
            items[i]['entity_kind'] = self.entities[items[i]['entity_kind']]
        return items
    
    def parse(self, item, **kwargs):
        return self.translate_entity(self.parser.parse(item))
    
    def get_all_grammar_entities(self):
        return list(self.entities.values())

In [55]:
parser = BuiltinEntityTagger()
parser.parse("Buy milk for 3.5 dollars")

[{'value': '3.5 dollars',
  'range': {'start': 13, 'end': 24},
  'entity': {'kind': 'AmountOfMoney',
   'value': 3.5,
   'precision': 'Exact',
   'unit': '$'},
  'alternatives': [],
  'entity_kind': '@amountOfMoney'}]

In [6]:
import importlib

In [19]:
os.path.dirname(os.getcwd())

'/home/h4zzkr/lab/delOS'

In [25]:
mymodule = importlib.import_module(f'core.nlu.semantic_taggers.BuiltinTagger.tagger')

In [5]:
module = __import__('/home/h4zzkr/lab/delOS/core/semantic_taggers/BuiltinTagger/tagger')

ModuleNotFoundError: No module named '/home/h4zzkr/lab/delOS/core/semantic_taggers/BuiltinTagger/tagger'